In [197]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    StaleElementReferenceException,
    WebDriverException,
    ElementClickInterceptedException,
)
from datetime import datetime, timedelta 
import time, re
import pandas as pd

In [198]:

#MAX_PAGE = int(Variable.get("MAX_PAGE"))
MAX_PAGE = 3
DRIVER_PATH = "/Users/kim-youngho/git/GeoNewsApt/notebook/chromedriver-mac-arm64/chromedriver"
# 오늘 날짜 문자열 "YYYYMMDD"
today_str = datetime.today().strftime("%Y%m%d")

In [199]:
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

try:
    service = Service(executable_path=DRIVER_PATH)
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 5)
except WebDriverException as e:
    print(f"[FATAL] Failed to initialize WebDriver: {e}")

article_links = list()
cutoff_date = 20200901  # 기준 날짜

In [200]:
article = {}
url = 'https://news.kbs.co.kr/news/pc/category/category.do?ctcd=0004&ref=pSiteMap#20250901&1'

In [201]:
driver.get(url)

In [202]:
next_btn = driver.find_element(By.CSS_SELECTOR, '#contents > div.box.padding-24.field-contents-wrapper.category-main-list > div.common-pagination > button.next-button')

In [203]:
while True:
    try:
        time.sleep(4)
        # 5초 내에 next_btn이 클릭 가능해지면 클릭
        next_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "#contents > div.box.padding-24.field-contents-wrapper.category-main-list > div.common-pagination > button.next-button"))
        )
        next_btn.click()
        container = driver.find_element(By.CSS_SELECTOR, "#contents > div.box.padding-24.field-contents-wrapper.category-main-list > div.box-contents.has-wrap")
        links = container.find_elements(By.TAG_NAME, "a")
        
        for link in links:
            href = link.get_attribute("href")
            if href:
                article_links.append(href)
    except (ElementClickInterceptedException, TimeoutException):
        print("마지막 페이지에 도달했습니다.")
        break

print(f"article_links is {article_links}")
article = {}

for url in article_links:
    driver.get(url)
    
    # 기사 내용
    content = driver.find_element(By.CSS_SELECTOR, '#cont_newstext').text
    
    # 날짜
    date_text = driver.find_element(
        By.CSS_SELECTOR, 
        '#contents > div > div.view-contents-wrapper > div.view-headline.view-box > div.dates > em.input-date'
    ).text
    
    m = re.search(r'(\d{4})\.(\d{2})\.(\d{2})', date_text)
    if m:
        date_str = f"{m.group(1)}-{m.group(2)}-{m.group(3)}"
    else:
        date_str = None  # 날짜 정보 없으면 None 처리
    
    # 내용이 있는 경우만 저장
    if content.strip():
        article[url] = {
            'date': date_str,
            'content': content,
            'publisher': 'KSB'
        }

마지막 페이지에 도달했습니다.
article_links is ['https://news.kbs.co.kr/news/view.do?ncd=8345744', 'https://news.kbs.co.kr/news/view.do?ncd=8345702', 'https://news.kbs.co.kr/news/view.do?ncd=8345686', 'https://news.kbs.co.kr/news/view.do?ncd=8345663', 'https://news.kbs.co.kr/news/view.do?ncd=8345662', 'https://news.kbs.co.kr/news/view.do?ncd=8345619', 'https://news.kbs.co.kr/news/view.do?ncd=8345605', 'https://news.kbs.co.kr/news/view.do?ncd=8345596', 'https://news.kbs.co.kr/news/view.do?ncd=8345572', 'https://news.kbs.co.kr/news/view.do?ncd=8345521', 'https://news.kbs.co.kr/news/view.do?ncd=8345518', 'https://news.kbs.co.kr/news/view.do?ncd=8345512', 'https://news.kbs.co.kr/news/view.do?ncd=8345501', 'https://news.kbs.co.kr/news/view.do?ncd=8345495', 'https://news.kbs.co.kr/news/view.do?ncd=8345487', 'https://news.kbs.co.kr/news/view.do?ncd=8345485', 'https://news.kbs.co.kr/news/view.do?ncd=8345484', 'https://news.kbs.co.kr/news/view.do?ncd=8345472', 'https://news.kbs.co.kr/news/view.do?ncd=834545

In [204]:
len(article_links)

48

In [205]:
article

{'https://news.kbs.co.kr/news/view.do?ncd=8345744': {'date': '2025-09-01',
  'content': '롯데카드가 해킹 공격을 당한 사실이 뒤늦게 확인됐습니다.\n\n롯데카드는 최근 악성코드 형태의 해킹 공격을 당한 사실을 오늘(1일) 금융감독원에 신고했습니다.\n\n롯데카드는 지난달 26일 서버를 점검하던 중 일부 서버가 악성코드에 감염된 사실을 확인했다고 밝혔습니다.\n\n이후 전체 서버를 정밀 조사했고, 서버 3개에서 악성코드 2종과 웹쉘 5종을 발견했다고 덧붙였습니다.\n\n웹셀(Web Shell)은 해커가 공격에 성공한 뒤 서버에 업로드하거나 심어두는 악성 스크립트로 추가 공격 등에 활용할 수 있습니다.\n\n어제(31일) 온라인 결제 서버에서 자료를 유출해 가려는 흔적이 확인했고, 롯데카드는 오늘 금융당국에 신고했습니다.\n\n롯데카드는 외부 조사업체를 통해 정밀 조사를 진행했으나 현재까지는 주요 정보의 외부 유출이나 랜섬웨어와 같은 심각한 악성코드 감염은 확인되지 않았다고 설명했습니다.\n\n롯데카드는 올해 상반기 기준으로 960여만명이 가입해 있습니다.',
  'publisher': 'KSB'},
 'https://news.kbs.co.kr/news/view.do?ncd=8345702': {'date': '2025-09-01',
  'content': '부산 가덕도신공항 부지 조성 공사를 위한 전문가 간담회가 이번 주 진행될 예정입니다.\n\n국토교통부 등에 따르면, 오는 4일 오전 가덕도신공항건설공단은 부산에 있는 본사에서 ‘부지 조성 공사 적정공기 관련 토론회’를 엽니다.\n\n공단은 이 자리에서 공단이 산정한 적정공기 검토 내용을 공개하고 이에 대한 의견을 들을 예정인 것으로 전해졌습니다.\n\n다만, 이번 토론회는 비공개로 진행됩니다.\n\n국토부 관계자는 “정상화 방안 마련을 위한 전문가 회의는 계속 진행 중이다”면서 “관련 방안이 확정되기 전까지는 비공개로 진행하고 있다”고 밝혔습

In [206]:
len(article)

42

In [207]:
driver.quit()